In [ ]:
import backtrader as bt
import pandas as pd
import numpy as np

# 讀取數據並處理日期列
data = pd.read_csv('小台指.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# 創建 PandasData 類
class PandasData(bt.feeds.PandasData):
    lines = ('datetime',)
    params = (
        ('datetime', None),
        ('open', 'Open'),
        ('high', 'High'),
        ('low', 'Low'),
        ('close', 'Close'),
        ('volume', 'Volume'),
        ('openinterest', None),
    )

# 轉換數據
data_feed = PandasData(dataname=data)

# 定義 MACD 策略
class MACDStrategy(bt.SignalStrategy):
    def __init__(self, short_period, long_period):
        self.short_ema = bt.ind.EMA(self.data.close, period=short_period)
        self.long_ema = bt.ind.EMA(self.data.close, period=long_period)
        self.macd = self.short_ema - self.long_ema
        self.signal = bt.ind.EMA(self.macd, period=9)
        self.crossover = bt.ind.CrossOver(self.macd, self.signal)
    
    def next(self):
        if self.crossover > 0:
            if not self.position:
                self.buy()
        elif self.crossover < 0:
            if self.position:
                self.sell()

# 回測策略
def backtest_strategy(short_period, long_period, data_feed):
    cerebro = bt.Cerebro()
    cerebro.addstrategy(MACDStrategy, short_period=short_period, long_period=long_period)
    cerebro.adddata(data_feed)
    cerebro.run()
    return cerebro.broker.getvalue()

# 優化參數
best_return = -np.inf
best_params = (None, None)

for short_period in range(1, 51):
    for long_period in range(short_period + 1, 101):
        return_value = backtest_strategy(short_period, long_period, data_feed)
        if return_value > best_return:
            best_return = return_value
            best_params = (short_period, long_period)

print(f"Best parameters: Short EMA = {best_params[0]}, Long EMA = {best_params[1]}")
print(f"Best return: {best_return}")

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import itertools

# 计算 MACD 的函数
def calculate_MACD(data, short_window, long_window, signal_window):
    # 计算短期和长期的 EMA
    data['EMA_short'] = data['Close'].ewm(span=short_window, adjust=False).mean()
    data['EMA_long'] = data['Close'].ewm(span=long_window, adjust=False).mean()
    # 计算 MACD 线
    data['MACD_line'] = data['EMA_short'] - data['EMA_long']
    # 计算信号线
    data['Signal_line'] = data['MACD_line'].ewm(span=signal_window, adjust=False).mean()
    return data

# MACD 交易策略回测函数
def backtest_MACD(data, short_window, long_window, signal_window):
    data = data.copy()  # 避免修改原始数据
    data = calculate_MACD(data, short_window, long_window, signal_window)
    data['Signal'] = 0  # 初始信号为 0

    # 生成交易信号
    # 当 MACD 线从下方穿过信号线，产生买入信号（多头）
    data['Signal'] = np.where(
        (data['MACD_line'].shift(1) < data['Signal_line'].shift(1)) &
        (data['MACD_line'] >= data['Signal_line']), 1, data['Signal'])
    # 当 MACD 线从上方穿过信号线，产生卖出信号（空头）
    data['Signal'] = np.where(
        (data['MACD_line'].shift(1) > data['Signal_line'].shift(1)) &
        (data['MACD_line'] <= data['Signal_line']), -1, data['Signal'])

    initial_capital = 200000  # 初始资金
    capital = initial_capital
    margin_per_unit = 73000   # 每单位保证金
    max_margin_loss = margin_per_unit * 0.7  # 当损失达到保证金的 70% 时平仓
    position = 0  # 持仓状态：1 为多头，-1 为空头，0 为空仓
    entry_price = 0  # 建仓价格
    total_pnl = 0    # 累计盈亏
    capital_history = []  # 用于记录每日资金变化

    data = data.dropna().reset_index()  # 重置索引，便于迭代

    for i in range(1, len(data)):
        # 获取当日数据
        price_today = data.loc[i, 'Close']
        price_yesterday = data.loc[i - 1, 'Close']
        signal = data.loc[i, 'Signal']

        # 更新持仓盈亏
        if position != 0:
            price_change = price_today - price_yesterday
            pnl = position * price_change * 50  # 每点 50 新台币
            total_pnl += pnl
            capital += pnl

            # 计算持仓的浮动盈亏
            position_pnl = position * (price_today - entry_price) * 50

            # 检查止损条件
            if position_pnl <= -max_margin_loss:
                # 平仓
                position = 0
                entry_price = 0
                capital += margin_per_unit  # 返还保证金

        # 检查交易信号
        if position == 0:
            if signal != 0:
                # 检查是否有足够的资金支付保证金
                if capital >= margin_per_unit:
                    # 建仓
                    position = signal
                    entry_price = price_today
                    capital -= margin_per_unit  # 扣除保证金
        elif signal != 0 and signal != position:
            # 反向信号，平仓并反手
            # 平仓
            capital += margin_per_unit  # 返还保证金
            # 检查是否有足够的资金支付新保证金
            if capital >= margin_per_unit:
                # 建立新仓位
                position = signal
                entry_price = price_today
                capital -= margin_per_unit  # 扣除保证金
            else:
                # 资金不足，无法建立新仓位
                position = 0
                entry_price = 0

        capital_history.append(capital)

    # 回测结束后，平掉所有持仓
    if position != 0:
        # 计算最终持仓盈亏
        price_change = data.loc[len(data) - 1, 'Close'] - entry_price
        pnl = position * price_change * 50
        total_pnl += pnl
        capital += pnl
        capital += margin_per_unit  # 返还保证金
        position = 0
        entry_price = 0

    cumulative_return = capital - initial_capital  # 计算累计收益

    return cumulative_return

# 设置标的及日期
ticker = '^TWII'  # 台湾加权指数
start_date = '2000-01-01'
end_date = '2023-12-31'

# 下载数据
data = yf.download(ticker, start=start_date, end=end_date)

# 确保数据下载正确
if data.empty:
    print("数据下载失败，请检查 ticker 代码或日期范围。")
else:
    # 定义参数范围
    short_window_range = range(1, 31, 1)      # 短期 EMA 周期从 1 到 30
    long_window_range = range(60, 91, 1)      # 长期 EMA 周期从 60 到 90
    signal_window_range = range(5, 21, 1)     # 信号线 EMA 周期从 5 到 20

    # 生成所有参数组合
    parameter_combinations = list(itertools.product(short_window_range, long_window_range, signal_window_range))

    # 存储结果的列表
    results = []

    # 迭代所有参数组合
    total_combinations = len(parameter_combinations)
    print(f"总共需要测试的参数组合数量：{total_combinations}")

    for idx, (short_window, long_window, signal_window) in enumerate(parameter_combinations, 1):
        # 确保短期窗口小于长期窗口，避免不合理的策略
        if short_window >= long_window:
            continue

        cumulative_return = backtest_MACD(data, short_window=short_window, long_window=long_window, signal_window=signal_window)

        results.append({
            'short_window': short_window,
            'long_window': long_window,
            'signal_window': signal_window,
            'cumulative_return': cumulative_return
        })

        # 进度提示（可选）
        if idx % 1000 == 0 or idx == total_combinations:
            print(f"已完成 {idx}/{total_combinations} 个参数组合的测试...")

    # 按累计收益从高到低排序结果
    sorted_results = sorted(results, key=lambda x: x['cumulative_return'], reverse=True)

    # 输出收益最高的参数组合
    best_result = sorted_results[0]
    print("\n收益最高的参数组合：")
    print(f"短期 EMA 周期: {best_result['short_window']}, 长期 EMA 周期: {best_result['long_window']}, 信号线 EMA 周期: {best_result['signal_window']}")
    print(f"最大累计收益: {best_result['cumulative_return']:.2f} 新台币")

    # 可选：输出收益最高的前 10 个参数组合
    print("\n收益最高的前 10 个参数组合：")
    for result in sorted_results[:10]:
        print(f"短期 EMA 周期: {result['short_window']}, 长期 EMA 周期: {result['long_window']}, 信号线 EMA 周期: {result['signal_window']}, 累计收益: {result['cumulative_return']:.2f} 新台币")


[*********************100%***********************]  1 of 1 completed


总共需要测试的参数组合数量：14880
已完成 1000/14880 个参数组合的测试...
已完成 2000/14880 个参数组合的测试...
已完成 3000/14880 个参数组合的测试...
已完成 4000/14880 个参数组合的测试...
已完成 5000/14880 个参数组合的测试...
已完成 6000/14880 个参数组合的测试...
已完成 7000/14880 个参数组合的测试...
已完成 8000/14880 个参数组合的测试...
已完成 9000/14880 个参数组合的测试...
已完成 10000/14880 个参数组合的测试...
已完成 11000/14880 个参数组合的测试...
已完成 12000/14880 个参数组合的测试...
已完成 13000/14880 个参数组合的测试...
已完成 14000/14880 个参数组合的测试...
已完成 14880/14880 个参数组合的测试...

收益最高的参数组合：
短期 EMA 周期: 1, 长期 EMA 周期: 88, 信号线 EMA 周期: 12
最大累计收益: 1434035.64 新台币

收益最高的前 10 个参数组合：
短期 EMA 周期: 1, 长期 EMA 周期: 88, 信号线 EMA 周期: 12, 累计收益: 1434035.64 新台币
短期 EMA 周期: 1, 长期 EMA 周期: 71, 信号线 EMA 周期: 12, 累计收益: 1432260.01 新台币
短期 EMA 周期: 1, 长期 EMA 周期: 66, 信号线 EMA 周期: 12, 累计收益: 1426926.86 新台币
短期 EMA 周期: 1, 长期 EMA 周期: 74, 信号线 EMA 周期: 12, 累计收益: 1425283.15 新台币
短期 EMA 周期: 1, 长期 EMA 周期: 67, 信号线 EMA 周期: 12, 累计收益: 1422286.91 新台币
短期 EMA 周期: 1, 长期 EMA 周期: 70, 信号线 EMA 周期: 12, 累计收益: 1420088.04 新台币
短期 EMA 周期: 1, 长期 EMA 周期: 84, 信号线 EMA 周期: 11, 累计收益: 1412241.02 新台币
短期 EMA 周期:

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import itertools

# 计算 MACD 的函数
def calculate_MACD(data, short_window, long_window, signal_window):
    # 计算短期和长期的 EMA
    data['EMA_short'] = data['Close'].ewm(span=short_window, adjust=False).mean()
    data['EMA_long'] = data['Close'].ewm(span=long_window, adjust=False).mean()
    # 计算 MACD 线
    data['MACD_line'] = data['EMA_short'] - data['EMA_long']
    # 计算信号线
    data['Signal_line'] = data['MACD_line'].ewm(span=signal_window, adjust=False).mean()
    return data

# MACD 交易策略回测函数，增加了交易手续费、交易次数、胜率和最大回撤的计算
def backtest_MACD(data, short_window, long_window, signal_window):
    data = data.copy()  # 避免修改原始数据
    data = calculate_MACD(data, short_window, long_window, signal_window)
    data['Signal'] = 0  # 初始信号为 0

    # 生成交易信号
    # 当 MACD 线从下方穿过信号线，产生买入信号（多头）
    data['Signal'] = np.where(
        (data['MACD_line'].shift(1) < data['Signal_line'].shift(1)) &
        (data['MACD_line'] >= data['Signal_line']), 1, data['Signal'])
    # 当 MACD 线从上方穿过信号线，产生卖出信号（空头）
    data['Signal'] = np.where(
        (data['MACD_line'].shift(1) > data['Signal_line'].shift(1)) &
        (data['MACD_line'] <= data['Signal_line']), -1, data['Signal'])

    initial_capital = 200000  # 初始资金
    capital = initial_capital
    margin_per_unit = 73000   # 每单位保证金
    max_margin_loss = margin_per_unit * 0.7  # 当损失达到保证金的 70% 时平仓
    transaction_fee = 35  # 每次交易手续费
    position = 0  # 持仓状态：1 为多头，-1 为空头，0 为空仓
    entry_price = 0  # 建仓价格
    total_pnl = 0    # 累计盈亏
    capital_history = []  # 用于记录每日资金变化
    trade_count = 0       # 交易次数
    win_count = 0         # 胜利次数
    drawdown = 0          # 回撤
    peak_capital = capital  # 资金峰值

    data = data.dropna().reset_index()  # 重置索引，便于迭代

    for i in range(1, len(data)):
        # 获取当日数据
        price_today = data.loc[i, 'Close']
        price_yesterday = data.loc[i - 1, 'Close']
        signal = data.loc[i, 'Signal']

        # 更新持仓盈亏
        if position != 0:
            price_change = price_today - price_yesterday
            pnl = position * price_change * 50  # 每点 50 新台币
            total_pnl += pnl
            capital += pnl

            # 计算持仓的浮动盈亏
            position_pnl = position * (price_today - entry_price) * 50

            # 检查止损条件
            if position_pnl <= -max_margin_loss:
                # 记录交易结果
                trade_count += 1
                if position_pnl > 0:
                    win_count += 1
                # 平仓
                position = 0
                entry_price = 0
                capital += margin_per_unit  # 返还保证金
                capital -= transaction_fee  # 扣除平仓手续费

        # 检查交易信号
        if position == 0:
            if signal != 0:
                # 检查是否有足够的资金支付保证金和手续费
                if capital >= margin_per_unit + transaction_fee:
                    # 建仓
                    position = signal
                    entry_price = price_today
                    capital -= margin_per_unit  # 扣除保证金
                    capital -= transaction_fee  # 扣除建仓手续费
        elif signal != 0 and signal != position:
            # 反向信号，平仓并反手
            # 记录交易结果
            trade_count += 1
            trade_pnl = position * (price_today - entry_price) * 50
            if trade_pnl > 0:
                win_count += 1
            # 平仓
            capital += margin_per_unit  # 返还保证金
            capital -= transaction_fee  # 扣除平仓手续费
            # 检查是否有足够的资金支付新保证金和手续费
            if capital >= margin_per_unit + transaction_fee:
                # 建立新仓位
                position = signal
                entry_price = price_today
                capital -= margin_per_unit  # 扣除保证金
                capital -= transaction_fee  # 扣除建仓手续费
            else:
                # 资金不足，无法建立新仓位
                position = 0
                entry_price = 0

        # 更新资金峰值和回撤
        if capital > peak_capital:
            peak_capital = capital
        drawdown = max(drawdown, peak_capital - capital)

        capital_history.append(capital)

    # 回测结束后，平掉所有持仓
    if position != 0:
        # 计算最终持仓盈亏
        price_change = data.loc[len(data) - 1, 'Close'] - entry_price
        pnl = position * price_change * 50
        total_pnl += pnl
        capital += pnl
        # 记录交易结果
        trade_count += 1
        if pnl > 0:
            win_count += 1
        capital += margin_per_unit  # 返还保证金
        capital -= transaction_fee  # 扣除平仓手续费
        position = 0
        entry_price = 0

    cumulative_return = capital - initial_capital  # 计算累计收益

    # 计算胜率
    if trade_count > 0:
        win_rate = win_count / trade_count
    else:
        win_rate = 0

    # 计算最大回撤
    max_drawdown = drawdown

    return cumulative_return, trade_count, win_rate, max_drawdown

# 设置标的及日期
ticker = '^TWII'  # 台湾加权指数
start_date = '2000-01-01'
end_date = '2023-12-31'

# 下载数据
data = yf.download(ticker, start=start_date, end=end_date)

# 确保数据下载正确
if data.empty:
    print("数据下载失败，请检查 ticker 代码或日期范围。")
else:
    # 定义参数范围
    short_window_range = range(1, 31, 1)      # 短期 EMA 周期从 1 到 30
    long_window_range = range(60, 91, 1)      # 长期 EMA 周期从 60 到 90
    signal_window_range = range(5, 21, 1)     # 信号线 EMA 周期从 5 到 20

    # 生成所有参数组合
    parameter_combinations = list(itertools.product(short_window_range, long_window_range, signal_window_range))

    # 存储结果的列表
    results = []

    # 迭代所有参数组合
    total_combinations = len(parameter_combinations)
    print(f"总共需要测试的参数组合数量：{total_combinations}")

    for idx, (short_window, long_window, signal_window) in enumerate(parameter_combinations, 1):
        # 确保短期窗口小于长期窗口，避免不合理的策略
        if short_window >= long_window:
            continue

        cumulative_return, trade_count, win_rate, max_drawdown = backtest_MACD(
            data, short_window=short_window, long_window=long_window, signal_window=signal_window)

        results.append({
            'short_window': short_window,
            'long_window': long_window,
            'signal_window': signal_window,
            'cumulative_return': cumulative_return,
            'trade_count': trade_count,
            'win_rate': win_rate,
            'max_drawdown': max_drawdown
        })

        # 进度提示（可选）
        if idx % 1000 == 0 or idx == total_combinations:
            print(f"已完成 {idx}/{total_combinations} 个参数组合的测试...")

    # 按累计收益从高到低排序结果
    sorted_results = sorted(results, key=lambda x: x['cumulative_return'], reverse=True)

    # 输出收益最高的参数组合
    best_result = sorted_results[0]
    print("\n收益最高的参数组合：")
    print(f"短期 EMA 周期: {best_result['short_window']}, 长期 EMA 周期: {best_result['long_window']}, 信号线 EMA 周期: {best_result['signal_window']}")
    print(f"最大累计收益: {best_result['cumulative_return']:.2f} 新台币")
    print(f"交易次数: {best_result['trade_count']}, 胜率: {best_result['win_rate']*100:.2f}%, 最大回撤: {best_result['max_drawdown']:.2f} 新台币")

    # 可选：输出收益最高的前 10 个参数组合
    print("\n收益最高的前 10 个参数组合：")
    for result in sorted_results[:10]:
        print(f"短期 EMA 周期: {result['short_window']}, 长期 EMA 周期: {result['long_window']}, 信号线 EMA 周期: {result['signal_window']}, "
              f"累计收益: {result['cumulative_return']:.2f} 新台币, 交易次数: {result['trade_count']}, "
              f"胜率: {result['win_rate']*100:.2f}%, 最大回撤: {result['max_drawdown']:.2f} 新台币")


[*********************100%***********************]  1 of 1 completed


总共需要测试的参数组合数量：14880
已完成 1000/14880 个参数组合的测试...
已完成 2000/14880 个参数组合的测试...
